<a href="https://colab.research.google.com/github/TacticalCoders/NLP/blob/main/Transformers_for_NLP/ch01/positional_encoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gensim==3.8.3
import torch
import nltk
nltk.download('punkt')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.2 MB 57.6 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
import math
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize 
import gensim 
from gensim.models import Word2Vec 
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings(action = 'ignore') 


dprint=0 # prints outputs if set to 1, default=0

#‘text.txt’ file 
sample = open("text.txt", "r") 
s = sample.read() 

# processing escape characters 
f = s.replace("\n", " ") 

data = [] 

# sentence parsing 
for i in sent_tokenize(f): 
	temp = [] 
	# tokenize the sentence into words 
	for j in word_tokenize(i): 
		temp.append(j.lower()) 
	data.append(temp) 

# Creating Skip Gram model 
#model2 = gensim.models.Word2Vec(data, min_count = 1, size = 512,window = 5, sg = 1) 
#model = Word2Vec(sentences=common_texts, vector_size=100, window=5, min_count=1, workers=4)
model2 = gensim.models.Word2Vec(data, min_count = 1, size = 512,window = 5, sg = 1)

# 1-The 2-black 3-cat 4-sat 5-on 6-the 7-couch 8-and 9-the 10-brown 11-dog 12-slept 13-on 14-the 15-rug.
word1='black'
word2='brown'
pos1=2
pos2=10
a=model2[word1]
b=model2[word2]

if(dprint==1):
        print(a)

# compute cosine similarity
dot = np.dot(a, b)
norma = np.linalg.norm(a)
normb = np.linalg.norm(b)
cos = dot / (norma * normb)

aa = a.reshape(1,512) 
ba = b.reshape(1,512)
cos_lib = cosine_similarity(aa, ba)

In [6]:
print(aa) # 'black' Word2Vec
print(ba) # 'brown' Word2Vec

[[-0.04496898  0.04651736  0.02040178 -0.0907924   0.01811739 -0.02693062
   0.04129254 -0.06010864 -0.01285995 -0.0018408  -0.05058975 -0.00707043
   0.00152775 -0.01322422  0.00636595  0.02874669 -0.11496101 -0.04711997
  -0.00057844  0.05045312  0.00970661 -0.00572899  0.05894233  0.02117172
  -0.03890822 -0.00935153 -0.01081965  0.0540962  -0.0008474   0.04401324
   0.03394766  0.03073468 -0.04720724 -0.05328015  0.02136751  0.09453347
   0.03061832  0.02551553 -0.02084323  0.04924917 -0.02790694 -0.00017648
   0.08688016  0.06084362  0.0268644   0.03034814  0.00784831 -0.02462878
  -0.02674808 -0.02609152 -0.00322859  0.04415093 -0.02183494 -0.0041774
  -0.05074496  0.0632149   0.06300684 -0.03928454 -0.07848868 -0.03023233
   0.0489714  -0.10740834 -0.02314817 -0.0281964  -0.03474345 -0.11798994
   0.01980542 -0.04461984 -0.00748411  0.02340221  0.01380571 -0.00110375
  -0.05514981 -0.04920753 -0.08628307 -0.06651565  0.02588722 -0.00999593
   0.01055522  0.06562412  0.00273212 -

사전 학습된 임베딩(Word2Vec)을 통해 'black'과 'brown'을 **512차원의 벡터**로 변환

In [8]:
print(cos_lib) # 두 단어 벡터의 코사인 유사도

[[0.999869]]


In [10]:
pe1=aa.copy()
pe2=aa.copy()
pe3=aa.copy()
paa=aa.copy()
pba=ba.copy()
d_model=512
max_print=d_model
max_length=20

for i in range(0, max_print,2):
                pe1[0][i] = math.sin(pos1 / (10000 ** ((2 * i)/d_model)))
                paa[0][i] = (paa[0][i]*math.sqrt(d_model))+ pe1[0][i]
                pe1[0][i+1] = math.cos(pos1 / (10000 ** ((2 * i)/d_model)))
                paa[0][i+1] = (paa[0][i+1]*math.sqrt(d_model))+pe1[0][i+1]
                if dprint==1:
                        print(i,pe1[0][i],i+1,pe1[0][i+1])
                        print(i,paa[0][i],i+1,paa[0][i+1])
                        print("\n")

#print(pe1)
# A  method in Pytorch using torch.exp and math.log :
max_len=max_length                
pe = torch.zeros(max_len, d_model)
position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
pe[:, 0::2] = torch.sin(position * div_term)
pe[:, 1::2] = torch.cos(position * div_term)
#print(pe[:, 0::2])

In [12]:
print(pe1) # pe1 -> 위치 인코딩 
print(paa) # paa -> 단어 임베딩 벡터에 위치 벡터를 더한 값

[[ 9.09297407e-01 -4.16146845e-01  9.58144367e-01 -2.86285430e-01
   9.87046242e-01 -1.60435960e-01  9.99164224e-01 -4.08766568e-02
   9.97479975e-01  7.09482506e-02  9.84703004e-01  1.74241230e-01
   9.63226616e-01  2.68690288e-01  9.35118318e-01  3.54335666e-01
   9.02130723e-01  4.31462824e-01  8.65725577e-01  5.00518918e-01
   8.27103794e-01  5.62049210e-01  7.87237823e-01  6.16649508e-01
   7.46903539e-01  6.64932430e-01  7.06710517e-01  7.07502782e-01
   6.67129099e-01  7.44942129e-01  6.28514349e-01  7.77797997e-01
   5.91127098e-01  8.06578398e-01  5.55151761e-01  8.31749082e-01
   5.20711303e-01  8.53732824e-01  4.87879932e-01  8.72910738e-01
   4.56693351e-01  8.89624178e-01  4.27157611e-01  9.04177189e-01
   3.99255812e-01  9.16839600e-01  3.72953892e-01  9.27849889e-01
   3.48205268e-01  9.37418282e-01  3.24954301e-01  9.45729733e-01
   3.03139359e-01  9.52946246e-01  2.82694966e-01  9.59209859e-01
   2.63553679e-01  9.64644730e-01  2.45647445e-01  9.69359219e-01
   2.28908

In [17]:
for i in range(0, max_print,2):
                pe2[0][i] = math.sin(pos2 / (10000 ** ((2 * i)/d_model)))
                pba[0][i] = (pba[0][i]*math.sqrt(d_model))+ pe2[0][i]
            
                pe2[0][i+1] = math.cos(pos2 / (10000 ** ((2 * i)/d_model)))
                pba[0][i+1] = (pba[0][i+1]*math.sqrt(d_model))+ pe2[0][i+1]
               
                if dprint==1:
                        print(i,pe2[0][i],i+1,pe2[0][i+1])
                        print(i,paa[0][i],i+1,paa[0][i+1])
                        print("\n")

print(word1,word2)
cos_lib = cosine_similarity(aa, ba)
print(cos_lib,"word similarity")
cos_lib = cosine_similarity(pe1, pe2)
print(cos_lib,"positional similarity")
cos_lib = cosine_similarity(paa, pba)
print(cos_lib,"positional encoding similarity")

print("\n--------------------------------------\n")

print(word1)
print("embedding")
print(aa)
print("positional encoding")
print(pe1)
print("encoded embedding")
print(paa)
print(word2)
print("embedding")
print(ba)
print("positional encoding")
print(pe2)
print("encoded embedding")
print(pba)

black brown
[[0.999869]] word similarity
[[0.8600013]] positional similarity
[[0.9588083]] positional encoding similarity

--------------------------------------

black
embedding
[[-0.04496898  0.04651736  0.02040178 -0.0907924   0.01811739 -0.02693062
   0.04129254 -0.06010864 -0.01285995 -0.0018408  -0.05058975 -0.00707043
   0.00152775 -0.01322422  0.00636595  0.02874669 -0.11496101 -0.04711997
  -0.00057844  0.05045312  0.00970661 -0.00572899  0.05894233  0.02117172
  -0.03890822 -0.00935153 -0.01081965  0.0540962  -0.0008474   0.04401324
   0.03394766  0.03073468 -0.04720724 -0.05328015  0.02136751  0.09453347
   0.03061832  0.02551553 -0.02084323  0.04924917 -0.02790694 -0.00017648
   0.08688016  0.06084362  0.0268644   0.03034814  0.00784831 -0.02462878
  -0.02674808 -0.02609152 -0.00322859  0.04415093 -0.02183494 -0.0041774
  -0.05074496  0.0632149   0.06300684 -0.03928454 -0.07848868 -0.03023233
   0.0489714  -0.10740834 -0.02314817 -0.0281964  -0.03474345 -0.11798994
   0.019